### Custom Loss Function Huber Loss

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

$ 2x - 1$

In [2]:
xs = np.array([-1.0,  0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys =  np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

In [3]:
model = keras.Sequential([keras.layers.Dense(units=1, input_shape=[1])])
model.compile(optimizer='sgd', loss='mean_squared_error')
model.fit(xs, ys, epochs=500, verbose=0)

print(model.predict([10.0]))

[[18.981144]]


In [4]:
def huber_loss(y_true, y_pred):
    threshold = 1
    error = y_true - y_pred
    is_small_error = tf.abs(error) <= threshold
    small_error_loss = tf.square(error) / 2
    big_error_loss = threshold * (tf.abs(error) - (0.5 * threshold))
    return tf.where(is_small_error, small_error_loss, big_error_loss)

In [5]:
model.compile(optimizer='sgd', loss=huber_loss)
model.fit(xs, ys, epochs=500, verbose=0)

print(model.predict([10.0]))

[[18.998579]]


#### Hyper Parameter Tuning with a Custom Loss Function

In [6]:
def huber_loss_with_threshold(threshold):
    def huber_loss(y_true, y_pred):
        threshold = 1
        error = y_true - y_pred
        is_small_error = tf.abs(error) <= threshold
        small_error_loss = tf.square(error) / 2
        big_error_loss = threshold * (tf.abs(error) - (0.5 * threshold))
        return tf.where(is_small_error, small_error_loss, big_error_loss)
    return huber_loss

In [7]:
model.compile(optimizer='sgd', loss=huber_loss_with_threshold(threshold=1))
model.fit(xs, ys, epochs=500, verbose=0)
model.predict([10.0])

array([[18.999895]], dtype=float32)

In [8]:
#### Create a Loss Function as a Class

from tensorflow.keras.losses import Loss

class HuberLoss(Loss): 
    threshold = 1
    def __init__(self, threshold): #Initialized object from the class
        super().__init__()
        self.threshold = threshold #Threshold is declare as a class variable which allow to give it an initial value
        
    def call(self, y_true, y_pred): #get excuted when an object is instanciated from the class HuberLoss
        error = y_true - y_pred
        is_small_error = tf.abs(error) <= self.threshold
        small_error_loss = tf.square(error) / 2
        big_error_loss = self.threshold * (tf.abs(error) - (0.5 * self.threshold))
        return tf.where(is_small_error, small_error_loss, big_error_loss)

In [9]:
model.compile(optimizer='sgd', loss=HuberLoss(threshold=0.88))
model.fit(xs, ys, epochs=500, verbose=0)
model.predict([10.0])

array([[18.999973]], dtype=float32)